In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [2]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  3% |


In [3]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
from torchmetrics import F1Score

In [4]:
os.chdir('/opt/ml/input/data')
os.getcwd()

'/opt/ml/input/data'

In [5]:
!find . -regex ".*\.\_[a-zA-Z0-9._]+" -delete

In [6]:
random_seed = 12
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [7]:
train_dir_path = '/opt/ml/input/data/train/'
train_image_path = '/opt/ml/input/data/train/images/'

dt_train = pd.read_csv(train_dir_path+'train.csv')
dt_train

,id,gender,race,age,path
0,1,female,Asian,45,000001_female_Asian_45
1,2,female,Asian,52,000002_female_Asian_52
2,4,male,Asian,54,000004_male_Asian_54
3,5,female,Asian,58,000005_female_Asian_58
4,6,female,Asian,59,000006_female_Asian_59
...,...,...,...,...,...
2695,6954,male,Asian,19,006954_male_Asian_19
2696,6955,male,Asian,19,006955_male_Asian_19
2697,6956,male,Asian,19,006956_male_Asian_19
2698,6957,male,Asian,20,006957_male_Asian_20


In [8]:
def get_age_range(age):
    if age < 30:
        return 0
    elif 30 <= age < 60:
        return 1
    else:
        return 2

In [9]:
def get_label(file, gender, age):
    if 'normal'in file:
        # Not Wear
        if gender =='male':
            if age < 30: 
                return 12
            elif age < 60:
                return 13
            else:
                return 14
        else: # female
            if age < 30:
                return 15
            elif age < 60:
                return 16
            else:
                return 17
    elif 'incorrect' in file:
        # Incorrect
        if gender == 'male':
            if age < 30: 
                return 6
            elif age < 60:
                return 7
            else:
                return 8
        else: # female
            if age < 30:
                return 9
            elif age < 60:
                return 10
            else:
                return 11
    else:
        # Wear
        if gender == 'male':
            if age < 30: 
                return 0
            elif age < 60:
                return 1
            else:
                return 2
        else: # female
            if age < 30:
                return 3
            elif age < 60:
                return 4
            else:
                return 5

In [10]:
dt_train['age_range'] = dt_train['age'].apply(lambda x : get_age_range(x))

In [11]:
dt_train['label'] = -1

for index, row in dt_train.iterrows():
    for file in os.listdir(os.path.join(train_dir_path, f"images/{row['path']}")):
        #print(get_label(file, row['gender'], row['age']))
        dt_train.at[index, 'label'] = get_label(file, row['gender'], row['age'])

In [12]:
dt_train

,id,gender,race,age,path,age_range,label
0,1,female,Asian,45,000001_female_Asian_45,1,4
1,2,female,Asian,52,000002_female_Asian_52,1,4
2,4,male,Asian,54,000004_male_Asian_54,1,1
3,5,female,Asian,58,000005_female_Asian_58,1,4
4,6,female,Asian,59,000006_female_Asian_59,1,4
...,...,...,...,...,...,...,...
2695,6954,male,Asian,19,006954_male_Asian_19,0,0
2696,6955,male,Asian,19,006955_male_Asian_19,0,0
2697,6956,male,Asian,19,006956_male_Asian_19,0,0
2698,6957,male,Asian,20,006957_male_Asian_20,0,0


In [13]:
# train_idx, valid_idx = train_test_split(np.arange(len(dt_train)),
#                                        test_size=0.2,
#                                        shuffle=True,
#                                        stratify=dt_train['age_range'])

In [14]:
# train_image = []
# train_label = []

# for idx in train_idx:
#     path = dt_train.iloc[idx]['path']
#     for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
#         train_image.append(train_image_path+path+'/'+file_name)
#         train_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))                                 

# print(train_label[0])

In [15]:
# valid_image = []
# valid_label = []

# for idx in valid_idx:
#     path = dt_train.iloc[idx]['path']
#     for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
#         valid_image.append(train_image_path+path+'/'+file_name)
#         valid_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))

In [16]:
from collections import Counter, defaultdict

def get_distribution(y_vals):
    y_distr = Counter(y_vals)
    y_vals_sum = sum(y_distr.values())
    return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]

In [17]:
from sklearn.model_selection import StratifiedKFold

groups = np.array(dt_train.id.values)

distrs = [get_distribution(dt_train.label)]
index = ['training set']

train_idx, valid_idx = [], []
train_groups, valid_groups = [], []
train_y, valid_y = [], []
num_fold = 4

# 4개의 폴드 세트로 분리하는 StratifiedKFold 세트 생성
skfold = StratifiedKFold(n_splits=num_fold, shuffle=True, random_state=2022)

for fold_index, (train_index, valid_index) in enumerate(skfold.split(X=dt_train.id, y=dt_train.label)):
    # print("%s %s" % (train_index, test_index))
    print(f"Fold {fold_index}, Training set : {len(train_index)}, Test set : {len(valid_index)} ")
    train_idx.append(train_index)
    valid_idx.append(valid_index)
    train_y.append(dt_train.label[train_index])
    valid_y.append(dt_train.label[valid_index])
    train_groups.append(groups[train_index])
    valid_groups.append(groups[valid_index])

    distrs.append(get_distribution(train_y[-1]))
    index.append(f"Fold {fold_index} - Training set")
    distrs.append(get_distribution(valid_y[-1]))
    index.append(f"Fold {fold_index} - Testing set")

    # print("train_groups: ", train_groups)
    # print("train_y: ",train_y)
    # print("test_groups: ",valid_groups)
    # print("test_y: ",valid_y)
    print("-------------------------------------------------------------")

Fold 0, Training set : 2025, Test set : 675 
-------------------------------------------------------------
Fold 1, Training set : 2025, Test set : 675 
-------------------------------------------------------------
Fold 2, Training set : 2025, Test set : 675 
-------------------------------------------------------------
Fold 3, Training set : 2025, Test set : 675 
-------------------------------------------------------------


In [18]:
print('Distribution per class:')
pd.DataFrame(distrs, index=index, columns=[f'Label {l}' for l in range(np.max(train_y) + 1)])

Distribution per class:


,Label 0,Label 1,Label 2,Label 3,Label 4,Label 5,Label 6,Label 7,Label 8,Label 9,Label 10,Label 11,Label 12,Label 13,Label 14,Label 15
training set,17.59%,14.78%,3.52%,25.30%,30.00%,4.22%,1.19%,0.00%,0.00%,0.70%,0.00%,0.00%,1.81%,0.00%,0.00%,0.89%
Fold 0 - Training set,17.63%,14.77%,3.51%,25.28%,29.98%,4.25%,1.19%,0.00%,0.00%,0.74%,0.00%,0.00%,1.78%,0.00%,0.00%,0.89%
Fold 0 - Testing set,17.48%,14.81%,3.56%,25.33%,30.07%,4.15%,1.19%,0.00%,0.00%,0.59%,0.00%,0.00%,1.93%,0.00%,0.00%,0.89%
Fold 1 - Training set,17.58%,14.81%,3.56%,25.28%,29.98%,4.20%,1.19%,0.00%,0.00%,0.69%,0.00%,0.00%,1.83%,0.00%,0.00%,0.89%
Fold 1 - Testing set,17.63%,14.67%,3.41%,25.33%,30.07%,4.30%,1.19%,0.00%,0.00%,0.74%,0.00%,0.00%,1.78%,0.00%,0.00%,0.89%
Fold 2 - Training set,17.58%,14.77%,3.51%,25.33%,30.02%,4.20%,1.19%,0.00%,0.00%,0.69%,0.00%,0.00%,1.83%,0.00%,0.00%,0.89%
Fold 2 - Testing set,17.63%,14.81%,3.56%,25.19%,29.93%,4.30%,1.19%,0.00%,0.00%,0.74%,0.00%,0.00%,1.78%,0.00%,0.00%,0.89%
Fold 3 - Training set,17.58%,14.77%,3.51%,25.28%,30.02%,4.25%,1.19%,0.00%,0.00%,0.69%,0.00%,0.00%,1.83%,0.00%,0.00%,0.89%
Fold 3 - Testing set,17.63%,14.81%,3.56%,25.33%,29.93%,4.15%,1.19%,0.00%,0.00%,0.74%,0.00%,0.00%,1.78%,0.00%,0.00%,0.89%


In [19]:
def onehot_enc(x):
    def gender(i):
        if i == 'male':
            return 0
        elif i == 'female':
            return 3
    def age(j):
        j = int(j)
        if j < 30:
            return 0
        elif j >= 30 and j < 60:
            return 1
        elif j >= 60:
            return 2
    def mask(k):
        if k == 'normal':
            return 12
        elif 'incorrect' in k:
            return 6
        else:
            return 0
    return gender(x[0]) + age(x[1]) + mask(x[2])

In [20]:
# train_data = pd.Series(train_image)
# train_label = pd.Series(train_label)

# valid_data = pd.Series(valid_image)
# valid_label = pd.Series(valid_label)

In [21]:
class Dataset_Mask(Dataset):
    def __init__(self, data, label, encoding=True, midcrop=True, transform=None):
        self.encoding = encoding
        self.midcrop = midcrop
        self.data = data
        self.label = label
        self.transform = transform
        
        if encoding:
            self.label = self.label.apply(onehot_enc)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        X = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        y = self.label[idx]
        
        if self.midcrop:
            X = X[70:420, 17:367]
        
        if self.transform:
            return self.transform(X), y
        return X, y

In [22]:
def make_class_weights(labels, class_num):
    # print("Labels shape:\n", labels.shape)
    # print("Given labels:\n", labels)

    labels = labels.apply(lambda x : x % class_num)

    labels = np.array(labels)
    class_weights = np.zeros_like(labels) 
    
    _, counts = np.unique(labels, return_counts=True)
    # 각 class가 몇 번 등장하는지 count

    # print("Labels:\n", labels)
    # print("Label count:\n", counts)

    for cls in range(class_num):
        class_weights = np.where(labels == cls, 1/counts[cls], class_weights)
        # label이 class에 해당하면 count의 역수 적용
    return class_weights

class_num = 18
age_class_num = 3

In [23]:
model = torchvision.models.resnet50(pretrained=True)
print('필요 입력 채널 개수', model.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', model.fc.weight.shape[0])
print(model)

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


필요 입력 채널 개수 3
네트워크 출력 채널 개수 1000
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256

In [24]:
import math

model.fc = nn.Linear(in_features=2048, out_features=class_num, bias=True)
nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

print('필요 입력 채널 개수', model.conv1.weight.shape[1])
print('네트워크 출력 채널 개수', model.fc.weight.shape[0])

필요 입력 채널 개수 3
네트워크 출력 채널 개수 18


In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"using {device}")

model.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 10

batch_size = 64

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

using cuda:0


In [26]:
best_val_acc = 0
best_val_loss = np.inf
best_f1_score = 0

def train(fold_idx, train_dataloader_mask, val_dataloader_mask):
    patience = 3
    cur_count = 0

    cur_val_acc = best_val_acc
    cur_val_loss = best_val_loss
    cur_f1_score = best_f1_score

    f1 = F1Score(num_classes=class_num, average='macro').to(device)

    for epoch in range(NUM_EPOCH):
        model.train()
        loss_value = 0
        matches = 0
        for train_batch in tqdm(train_dataloader_mask):
            inputs, labels = train_batch
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            loss = criterion(outs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if epoch % 5 == 0:
                torch.save(model, '/opt/ml/checkpoint/resnet50_kfold/checkpoint_%dfold_ep%d.pt'%(fold_idx, epoch))
            
            loss_value += loss.item()
            matches += (preds == labels).sum().item()

        train_loss = loss_value / batch_size
        train_acc = matches / batch_size

        loss_value = 0
        matches = 0

        print(f"epoch[{epoch}/{NUM_EPOCH}] training loss {train_loss:.3f}, training accuracy {train_acc:.3f}")
            
        with torch.no_grad():
            model.eval()
            val_loss_items = []
            val_acc_items = []
            f1_score = 0
            for val_batch in val_dataloader_mask:
                inputs, labels = val_batch
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                outs = model(inputs)
                preds = torch.argmax(outs, dim=-1)
                
                loss_item = criterion(outs, labels).item()
                acc_item = (labels==preds).sum().item()
                val_loss_items.append(loss_item)
                val_acc_items.append(acc_item)
                f1_score += f1(outs, labels)
                
            val_loss = np.sum(val_loss_items) / len(val_dataloader_mask)
            val_acc = np.sum(val_acc_items) / len(val_dataloader_mask.dataset)
            f1_score /= len(val_dataloader_mask)
            
            if val_loss < cur_val_loss:
                cur_val_loss = val_loss

            if val_acc > cur_val_acc:
                cur_val_acc = val_acc

            if f1_score > cur_f1_score:
                cur_f1_score = f1_score
                cur_count = 0
                torch.save(model, '/opt/ml/checkpoint/resnet50_kfold/checkpoint_best.pt')
                print("Update checkpoint!!!")
            else:
                cur_count += 1
                if cur_count >= patience:
                    print("Early Stopping!")
                    break
            print(f"[val] acc : {val_acc:.3f}, loss : {val_loss:.3f}, f1 score: {f1_score:.3f}")
            print(f"best acc : {cur_val_acc:.3f}, best loss : {cur_val_loss:.3f}, best f1 : {cur_f1_score:.3f}")

    return cur_val_acc, cur_val_loss, cur_f1_score

In [27]:
for fold_idx in range(num_fold):
    print("********", fold_idx,"th fold********")
    train_image = []
    train_label = []
    for idx in train_idx[fold_idx]:
        path = dt_train.iloc[idx]['path']
        for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
            train_image.append(train_image_path+path+'/'+file_name)
            train_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0])) 

    valid_image = []
    valid_label = []
    for idx in valid_idx[fold_idx]:
        path = dt_train.iloc[idx]['path']
        for file_name in [i for i in os.listdir(train_image_path+path) if i[0] != '.']:
            valid_image.append(train_image_path+path+'/'+file_name)
            valid_label.append((path.split('_')[1], path.split('_')[3], file_name.split('.')[0]))                                 

    train_data = pd.Series(train_image)
    train_label = pd.Series(train_label)
    valid_data = pd.Series(valid_image)
    valid_label = pd.Series(valid_label)

    mask_train_set = Dataset_Mask(data=train_data, label=train_label, transform = transforms.Compose([
                                transforms.ToTensor()
                            ]))

    mask_val_set = Dataset_Mask(data=valid_data, label=valid_label, transform = transforms.Compose([
                                    transforms.ToTensor()
                                ]))

    # print(f'training data size : {len(mask_train_set)}')
    # print(f'validation data size : {len(mask_val_set)}')
    
    class_weights = make_class_weights(mask_train_set.label, age_class_num)

    #sampler = sampler.WeightedRandomSampler(weights=class_weights, num_samples=len(class_weights))

    #train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, sampler=sampler, num_workers=2)
    train_dataloader_mask = DataLoader(dataset = mask_train_set, batch_size=batch_size, shuffle=True, num_workers=2)
    val_dataloader_mask = DataLoader(dataset = mask_val_set, batch_size=batch_size, shuffle=True,  num_workers=2)

    # print(len(train_dataloader_mask))
    # print(len(val_dataloader_mask))

    cur_val_acc, cur_val_loss, cur_f1_score = train(fold_idx, train_dataloader_mask, val_dataloader_mask)

    best_val_acc += cur_val_acc
    best_val_loss += cur_val_loss
    best_f1_score += cur_f1_score

    print("--------------------------------------------------------------------------------------------------------------------")
    print(f"{fold_idx}'th FOLD - best acc : {cur_val_acc:.3f}, best loss : {cur_val_loss:.3f}, best f1 : {cur_f1_score:.3f}")
    print("--------------------------------------------------------------------------------------------------------------------")

best_val_acc /= num_fold
best_val_loss /= num_fold
best_f1_score /= num_fold

print(f"****FINAL - best acc : {best_val_acc:.3f}, best loss : {best_val_loss:.3f}, best f1 : {best_f1_score:.3f}*****")

******** 0 th fold********


100%|██████████| 222/222 [04:00<00:00,  1.08s/it]

epoch[0/10] training loss 1.232, training accuracy 197.859



  0%|          | 0/222 [00:00<?, ?it/s]

Update checkpoint!!!
[val] acc : 0.880, loss : 0.389, f1 score: 0.758
best acc : 0.880, best loss : 0.389, best f1 : 0.758


...

epoch[4/10] training loss 0.016, training accuracy 221.219



  0%|          | 0/222 [00:00<?, ?it/s]

[val] acc : 0.963, loss : 0.129, f1 score: 0.943
best acc : 0.981, best loss : 0.050, best f1 : 0.959


  6%|▋         | 14/222 [00:15<03:47,  1.09s/it]


KeyboardInterrupt: 

In [ ]:
temp = 4
best_val_acc /= temp
best_val_loss /= temp
best_f1_score /= temp

print(f"****FINAL - best acc : {best_val_acc:.3f}, best loss : {best_val_loss:.3f}, best f1 : {best_f1_score:.3f}*****")

In [28]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir_path = '/opt/ml/input/data/eval/'
test_image_path = '/opt/ml/input/data/eval/images/'

model = torch.load('/opt/ml/checkpoint/resnet50_kfold/checkpoint_best.pt')
submission = pd.read_csv(test_dir_path+'info.csv')
submission.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,0
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,0
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,0
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,0
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [29]:
image_paths = [os.path.join(test_image_path, img_id) for img_id in submission.ImageID]
test_image = pd.Series(image_paths)

In [30]:
class Test_Dataset(Dataset):
    def __init__(self, midcrop=True, transform=None):
        self.midcrop = midcrop
        self.data = test_image
        self.transform = transform
        
    def __len__(self):
        return len(test_image)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.data[idx]), cv2.COLOR_BGR2RGB)
        
        if self.midcrop:
            img = img[64:448]
            
        if self.transform:
            img = self.transform(img)
            
        return img

In [31]:
dataset = Test_Dataset(transform = transforms.Compose([
                            transforms.ToTensor()
                        ]))

loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = model.to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm(loader):
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir_path, 'submission_resnet50_kfold.csv'), index=False)
print('test inference is done!')

100%|██████████| 197/197 [00:44<00:00,  4.39it/s]

test inference is done!
